In [9]:
import numpy as np
from tqdm import tqdm
import torch
from torchvision.transforms import v2

### Cosine similarity as a function of augmentation strength

In [2]:
number_of_hq_samples = 1000
cosine_sim = np.load("/app/model_weights/ms1m_cache/cosine_sim.npy")
labels = np.load("/app/datasets/ms1m/labels.npy")

In [3]:
seen_ids = []
hq_samples = []
cosine_sims = []
sample_count = 0
sorted_sim = np.sort(cosine_sim)[::-1]
id_sorted = np.argsort(cosine_sim)[::-1]
i = 0
while sample_count < number_of_hq_samples:
    sample_id = labels[id_sorted[i]]
    if sample_id not in seen_ids:
        hq_samples.append(id_sorted[i])
        sample_count += 1
        i += 1
        seen_ids.append(sample_id)
        cosine_sims.append(sorted_sim[i])
    else:
        i += 1

In [4]:
from omegaconf import OmegaConf
from face_lib.models.lightning_wrappers import ResNet
from face_lib.models.scf import SphereConfidenceFace, SoftmaxWeights
from face_lib.models.heads import SCFHead

conf = OmegaConf.load("/app/configs/train/train_scf.yaml")
model_trained = torch.load("/app/model_weights/scf_base.ckpt")
backbone = ResNet(
    resnet_name="iresnet50_normalized",
    weights="/app/model_weights/backbone/ms1mv3_arcface_r50/backbone.pth",
    learnable=False,
)
backbone.eval()
softmax_weights = SoftmaxWeights(
    softmax_weights_path="/app/model_weights/backbone/ms1mv3_arcface_r50/softmax_weight.pt",
    radius=64,
).softmax_weights
softmax_weights = torch.nn.functional.normalize(softmax_weights, p=2.0, dim=1)
# head = SCFHead(25088, 12544)
# scf_model = SphereConfidenceFace(backbone, head, None, None, None, softmax_weights, None)

# scf_model.load_state_dict(model_trained["state_dict"])

/tmp/ipykernel_38615/992649674.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_trained = torch.load(
/app/face_lib/models/lightning_wrappers.py:38: FutureWarning: 

In [5]:
from face_lib.dataset_classes.lightning_datasets import MXFaceDataset

ms1m = MXFaceDataset(root_dir="/app/datasets/ms1m/", test=True)

/home/l.erlygin/miniconda/lib/python3.9/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [10]:
transform = v2.Compose(
    [
        v2.ToPILImage(),
        v2.ToTensor(),
        v2.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    ]
)

/home/l.erlygin/miniconda/lib/python3.9/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [19]:
image = transform(ms1m[hq_samples[-1]])

In [20]:
features = backbone(image[None, ...])["feature"][0, :]

/app/face_lib/models/iresnet.py:198: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self.fp16):


In [21]:
features @ softmax_weights[seen_ids[-1]]

tensor(0.9006, grad_fn=<DotBackward0>)

In [22]:
cosine_sims[-1]

0.9006073

In [7]:
from face_lib.models.scf import SphereConfidenceFace
emb_model = 

SyntaxError: invalid syntax (4100470065.py, line 2)

/app/face_lib/models/scf.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.softmax_weights = torch.load(softmax_weights_path)


In [20]:
softmax_weights.shape

torch.Size([85742, 512])